In [ ]:
import os
import numpy as np
import torch
from nilearn import plotting
import data
import visualize
from words import make_classifications, Predictions
from data import normalize_fmri_data, unnormalize_fmri_data, analyze_results
from LEM import extract_data_features, predAccuracy
from numpy.linalg import norm
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree, svm
from sklearn.neural_network import MLPRegressor

In [ ]:
class argObj:
    def __init__(self, data_dir, parent_submission_dir, subj):
        self.subj = format(subj, '02')
        self.data_dir = os.path.join(data_dir, 'subj' + self.subj)
        self.parent_submission_dir = parent_submission_dir
        self.subject_submission_dir = os.path.join(self.parent_submission_dir,
                                                   'subj' + self.subj)



In [ ]:
words2 = ['furniture', 'food', 'kitchenware', 'appliance', 'person', 'animal', 'vehicle', 'objects',
            'electronics', 'sports', 'traffic', 'outdoor', 'home', 'plumbing', 'computer']

In [ ]:
platform = 'jupyter_notebook'
device = 'cpu'
# device = torch.device('cpu')
# setting up the directories and ARGS
data_dir = ''#../MQP/algonauts_2023_challenge_data/'
parent_submission_dir = '../submission'
subj = 1 # @param ["1", "2", "3", "4", "5", "6", "7", "8"] {type:"raw", allow-input: true}
# args

args = argObj(data_dir, parent_submission_dir, subj)
fmri_dir = os.path.join(args.data_dir, 'training_split', 'training_fmri')
lh_fmri = np.load(os.path.join(fmri_dir, 'lh_training_fmri.npy'))
rh_fmri = np.load(os.path.join(fmri_dir, 'rh_training_fmri.npy'))

words = ['furniture', 'food', 'kitchenware', 'appliance', 'person', 'animal', 'vehicle', 'accessory',
            'electronics', 'sports', 'traffic', 'outdoor', 'home', 'clothing', 'hygiene', 'toy', 'plumbing', 
             'computer', 'fruit', 'vegetable', 'tool']


In [ ]:
print("________ Process Data ________")

# Normalize Data Before Split
lh_fmri, lh_data_min, lh_data_max = normalize_fmri_data(lh_fmri)
rh_fmri, rh_data_min, rh_data_max = normalize_fmri_data(rh_fmri)

print('LH training fMRI data shape:')
print(lh_fmri.shape)
print('(Training stimulus images × LH vertices)')

print('\nRH training fMRI data shape:')
print(rh_fmri.shape)
print('(Training stimulus images × RH vertices)')

train_img_dir = os.path.join(args.data_dir, 'training_split', 'training_images')
test_img_dir = os.path.join(args.data_dir, 'test_split', 'test_images')
train_img_list = os.listdir(train_img_dir)
train_img_list = train_img_list[: 4500]
train_img_list.sort()
test_img_list = os.listdir(test_img_dir)
test_img_list.sort()

print('\nTraining images: ' + str(len(train_img_list)))
print('\nTest images: ' + str(len(test_img_list)))


In [ ]:
print("________ Split Data ________")

idxs_train, idxs_val, idxs_test = data.splitdata(train_img_list, test_img_list, train_img_dir)
lh_fmri_train = lh_fmri[idxs_train]
rh_fmri_train = rh_fmri[idxs_train]
lh_fmri_val = lh_fmri[idxs_val]
rh_fmri_val = rh_fmri[idxs_val]

In [ ]:
print("________ Make Lists ________")

train_images = data.makeList(train_img_dir, train_img_list, idxs_train)
val_images = data.makeList(train_img_dir, train_img_list, idxs_val)
test_images = data.makeList(test_img_dir, test_img_list, idxs_test)
torch.cuda.empty_cache()

In [ ]:
print("________ Make Classifications ________")
lh_classifications = make_classifications(train_images, idxs_train, device)
rh_classifications = lh_classifications
lh_classifications_val = make_classifications(val_images, idxs_val, device)
rh_classifications_val = lh_classifications_val
lh_classifications_test = make_classifications(test_images, idxs_test, device)
rh_classifications_test = lh_classifications_test
torch.cuda.empty_cache()

In [ ]:
print("________ Extract Image Features ________")

train_imgs_dataloader, val_imgs_dataloader, test_imgs_dataloader = (
    data.transformData(train_img_dir, test_img_dir, idxs_train, idxs_val, idxs_test, 128))

features_train, features_val, features_test = (
    extract_data_features(train_imgs_dataloader, val_imgs_dataloader, test_imgs_dataloader, 128))

In [ ]:
print("________ Organize Input________")

LH_train_class, LH_train_FMRI = data.organize_input(lh_classifications, features_train, lh_fmri_train)
LH_val_class, LH_val_FMRI = data.organize_input(lh_classifications_val, features_val, lh_fmri_val)    
RH_train_class, RH_train_FMRI = data.organize_input(rh_classifications, features_train, rh_fmri_train)
RH_val_class, RH_val_FMRI = data.organize_input(rh_classifications_val, features_val, rh_fmri_val)


Linear Regression

In [ ]:
print("________ Predictions Validation ________")
model = LinearRegression()
lh_fmri_val_pred = Predictions(LH_train_class, LH_train_FMRI, LH_val_class, model)
print('checkpoint')
rh_fmri_val_pred = Predictions(RH_train_class, RH_train_FMRI, RH_val_class, model)
# lh_fmri_test_pred = Predictions(LH_train_class, LH_train_FMRI, LH_test_class )
# rh_fmri_test_pred = Predictions(RH_train_class, RH_train_FMRI, RH_test_class )


In [ ]:
print("________ Analyze Results ________")

analyze_results(LH_val_FMRI, lh_fmri_val_pred)
analyze_results(RH_val_FMRI, rh_fmri_val_pred)

MLP Regressor

In [ ]:
print("________ Predictions Validation ________")
model = MLPRegressor(random_state=1, max_iter=500)
model.fit(LH_train_class,LH_train_FMRI)
lh_fmri_val_pred = model.predict(LH_val_class)

model.fit(RH_train_class,RH_train_FMRI)
rh_fmri_val_pred = model.predict(RH_val_class)


In [ ]:
print("________ Analyze Results ________")
analyze_results(LH_val_FMRI, lh_fmri_val_pred)
analyze_results(RH_val_FMRI, rh_fmri_val_pred)

Decision Tree Regressor

In [ ]:
print("________ Predictions Validation ________")
model = tree.DecisionTreeRegressor()
lh_fmri_val_pred = Predictions(LH_train_class, LH_train_FMRI, LH_val_class, model)
print('checkpoint')
rh_fmri_val_pred = Predictions(RH_train_class, RH_train_FMRI, RH_val_class, model)
# lh_fmri_test_pred = Predictions(LH_train_class, LH_train_FMRI, LH_test_class )
# rh_fmri_test_pred = Predictions(RH_train_class, RH_train_FMRI, RH_test_class )

In [ ]:
print("________ Analyze Results ________")

analyze_results(LH_val_FMRI, lh_fmri_val_pred)
analyze_results(RH_val_FMRI, rh_fmri_val_pred)

In [ ]:
print("________ Unnormalize Data ________")

lh_fmri_val_pred = unnormalize_fmri_data(lh_fmri_val_pred, lh_data_min, lh_data_max)
rh_fmri_val_pred = unnormalize_fmri_data(rh_fmri_val_pred, rh_data_min, rh_data_max)

In [ ]:
print("________ Re-Load Data ________")
lh_fmri = np.load(os.path.join(fmri_dir, 'lh_training_fmri.npy'))
rh_fmri = np.load(os.path.join(fmri_dir, 'rh_training_fmri.npy'))

lh_fmri_val = lh_fmri[idxs_val]
rh_fmri_val = rh_fmri[idxs_val]


In [ ]:
print("________ Prediction Accuracy ________")

lh_correlation, rh_correlation = predAccuracy(lh_fmri_val_pred, lh_fmri_val, rh_fmri_val_pred, rh_fmri_val)


In [ ]:
print("________ Visualize Each Class ________")

length = len(words)
for clss in range(length):
    avg_lh_pred = []
    avg_lh_real = []
    avg_rh_pred = []
    avg_rh_real = []
    for i in range(len(lh_classifications_val)):

        if lh_classifications_val[i][1] == clss:
            avg_lh_pred.append(lh_fmri_val_pred[i])
            avg_lh_real.append(lh_fmri_val[i])

        if rh_classifications_val[i][1] == clss:
            avg_rh_pred.append(rh_fmri_val_pred[i])
            avg_rh_real.append(rh_fmri_val[i])
    # Only look at classes that are observed 
    if(len(avg_lh_pred) < 10):
        pass
    else:
        print(words[clss])
        lh = np.mean(avg_lh_pred, axis=0)
        rh = np.mean(avg_rh_pred, axis=0)
        print("MEAN PRED LH:\n", lh)
        print("MEAN PRED RH:\n", rh)
        visualize.plot_predictions(args, lh, rh, 'left')
        visualize.plot_predictions(args, lh, rh, 'right')
        plotting.show()
        lh2 = np.mean(avg_lh_real, axis=0)
        rh2 = np.mean(avg_rh_real, axis=0)
        print("MEAN REAL LH:\n", lh2)
        print("MEAN REAL RH:\n", rh2)
        visualize.plot_predictions(args, lh2, rh2, 'left')
        visualize.plot_predictions(args, lh2, rh2, 'right')
        plotting.show()
        corr = np.corrcoef(avg_lh_pred, avg_lh_real)
        print(len(avg_lh_pred))
        print("Corre ", np.mean(corr))

print("________ END ________")